In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import sys
sys.path.insert(1, "/home/oru2/project/project")

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import time
# import matplotlib.pyplot as plt
import attacks
from privacy_accountant import PrivacyAccountant
from tqdm import tqdm
from torchvision import datasets, transforms


In [18]:
use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")
batch_size = 100

np.random.seed(42)
torch.manual_seed(42)


## Dataloaders
train_dataset = datasets.MNIST('../mnist_data/', train=True, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))
test_dataset = datasets.MNIST('../mnist_data/', train=False, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
attack = attacks.PGD()


The history saving thread hit an unexpected error (OperationalError('unable to open database file')).History will not be written to the database.


In [6]:
import warnings
warnings.simplefilter("ignore")

MAX_GRAD_NORM = 1.2
EPSILON = 1
DELTA = 1e-5
EPOCHS = 20

LR = 1e-3
BATCH_SIZE = 100
MAX_PHYSICAL_BATCH_SIZE = 12

In [5]:
from model import fcNet
fc_model = fcNet(784, 128, 10).to(device)
num_epochs = 20
fc_model


fcNet(
  (layers): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): RNNLinear(in_features=784, out_features=128, bias=True)
    (2): ReLU(inplace=True)
    (3): RNNLinear(in_features=128, out_features=128, bias=True)
    (4): ReLU(inplace=True)
    (5): RNNLinear(in_features=128, out_features=10, bias=True)
  )
)

In [ ]:

criterion = nn.CrossEntropyLoss(reduction = 'none')
optimizer = optim.SGD(fc_model.parameters(), lr=LR)

In [5]:
from opacus.accountants import RDPAccountant
from opacus.utils.batch_memory_manager import BatchMemoryManager
n = 60000
# Create an RDP (Rényi Differential Privacy) accountant
accountant = RDPAccountant()
for _ in range(24000):
    accountant.step(noise_multiplier=1.1, sample_rate=100/60000)

accountant.get_epsilon(1e-5)

1.2331912114223702

In [16]:
from torch.nn.utils import clip_grad_norm_

noise_multiplier = 1.1
def train_dp(model, trainloader, optimizer, epoch, accountant, num_microbatches = 100):
    """
    Differentially Private version of the training procedure

    :param trainloader:
    :param model:
    :param optimizer:
    :param epoch:
    :return:
    """
    model.train()
    running_loss = 0.0
    for i, data in tqdm(enumerate(trainloader, 0), leave=True):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        running_loss += torch.mean(loss).item()
        
        losses = torch.mean(loss.reshape(num_microbatches, -1), dim=1)
        saved_var = dict()
        for tensor_name, tensor in model.named_parameters():
            saved_var[tensor_name] = torch.zeros_like(tensor)

        for j in losses:
            j.backward(retain_graph=True)
            torch.nn.utils.clip_grad_norm_(model.parameters(), MAX_GRAD_NORM)
            for tensor_name, tensor in model.named_parameters():
                new_grad = tensor.grad
                saved_var[tensor_name].add_(new_grad)
            model.zero_grad()

        for tensor_name, tensor in model.named_parameters():
            if device.type =='cuda':
                noise = torch.cuda.FloatTensor(tensor.shape).normal_(0, noise_multiplier)
            else:
                noise = torch.FloatTensor(tensor.shape).normal_(0, noise_multiplier)
            saved_var[tensor_name].add_(noise)
            tensor.grad = saved_var[tensor_name] / num_microbatches
        optimizer.step()
        accountant.step(noise_multiplier=1.1, sample_rate=100/60000)

        # if i > 0 and i % 20 == 0:
        #     print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            # plot(epoch * len(trainloader) + i, running_loss, 'Train Loss')
            # running_loss = 0.0
def train_dp_adv(model, trainloader, optimizer, epoch, accountant, num_microbatches = 100):
    """
    Differentially Private version of the training procedure

    :param trainloader:
    :param model:
    :param optimizer:
    :param epoch:
    :return:
    """
    model.train()
    running_loss = 0.0
    for i, data in tqdm(enumerate(trainloader, 0), leave=True):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        adversary_images = attack.pgd_untargeted(model, inputs, labels, 10, 0.1, 0.01).to(device)
        outputs = model(adversary_images)
        loss = criterion(outputs, labels)
        running_loss += torch.mean(loss).item()
        
        losses = torch.mean(loss.reshape(num_microbatches, -1), dim=1)
        saved_var = dict()
        for tensor_name, tensor in model.named_parameters():
            saved_var[tensor_name] = torch.zeros_like(tensor)

        for j in losses:
            j.backward(retain_graph=True)
            torch.nn.utils.clip_grad_norm_(model.parameters(), MAX_GRAD_NORM)
            for tensor_name, tensor in model.named_parameters():
                new_grad = tensor.grad
                saved_var[tensor_name].add_(new_grad)
            model.zero_grad()

        for tensor_name, tensor in model.named_parameters():
            if device.type =='cuda':
                noise = torch.cuda.FloatTensor(tensor.shape).normal_(0, noise_multiplier)
            else:
                noise = torch.FloatTensor(tensor.shape).normal_(0, noise_multiplier)
            saved_var[tensor_name].add_(noise)
            tensor.grad = saved_var[tensor_name] / num_microbatches
        optimizer.step()
        accountant.step(noise_multiplier=1.1, sample_rate=100/60000)

In [19]:
# train(fc_model, train_loader, optimizer, EPOCHS, device)
# for epoch in tqdm(range(EPOCHS), desc="Epoch", unit="epoch"):
#     train_dp(fc_model, train_loader, optimizer, epoch + 1, accountant)

ADV_EPOCHS = 20
for epoch in tqdm(range(ADV_EPOCHS), desc="Epoch", unit="epoch"):
    train_dp_adv(fc_model, train_loader, optimizer, epoch + 1, accountant)

600it [01:25,  7.02it/s]0/20 [00:00<?, ?epoch/s]
600it [01:25,  7.06it/s]1/20 [01:25<27:04, 85.50s/epoch]
600it [01:25,  7.05it/s]2/20 [02:50<25:33, 85.22s/epoch]
600it [01:24,  7.07it/s]3/20 [04:15<24:07, 85.16s/epoch]
600it [01:25,  7.06it/s]4/20 [05:40<22:40, 85.06s/epoch]
600it [01:25,  7.06it/s]5/20 [07:05<21:15, 85.05s/epoch]
600it [01:25,  7.05it/s]6/20 [08:30<19:50, 85.05s/epoch]
600it [01:24,  7.06it/s]7/20 [09:55<18:25, 85.06s/epoch]
600it [01:24,  7.10it/s]8/20 [11:20<17:00, 85.04s/epoch]
600it [01:24,  7.09it/s]9/20 [12:45<15:33, 84.87s/epoch]
600it [01:23,  7.18it/s]10/20 [14:09<14:07, 84.78s/epoch]
600it [01:24,  7.09it/s]11/20 [15:33<12:39, 84.42s/epoch]
600it [01:24,  7.08it/s]12/20 [16:58<11:15, 84.49s/epoch]
600it [01:24,  7.08it/s]13/20 [18:22<09:51, 84.56s/epoch]
600it [01:23,  7.15it/s]14/20 [19:47<08:27, 84.63s/epoch]
600it [01:24,  7.12it/s]15/20 [21:11<07:02, 84.43s/epoch]
600it [01:24,  7.11it/s]16/20 [22:35<05:37, 84.37s/epoch]
600it [01:23,  7.15it/s]17/20 [2

In [25]:
torch.save(fc_model, 'models/dp-adv.pt')

In [7]:
fc_model = torch.load("models/dp-adv.pt")

In [21]:
correct = 0
fc_model.eval()
for j, (images, labels) in enumerate(test_loader):
  images, labels = images.to(device), labels.to(device)
  logits = fc_model(images)
  _, preds = torch.max(logits, 1)
  correct += (preds == labels).sum().item()
  # print('Batch [{}/{}]'.format(j+1, len(test_loader)))
fc_model.train()
print('Accuracy = {}%'.format(float(correct) * 100 / 10000))

Accuracy = 80.06%


In [22]:
# from art.attacks.evasion.projected_gradient_descent.projected_gradient_descent_pytorch import ProjectedGradientDescentPyTorch

# pgd_attacker = ProjectedGradientDescentPyTorch(art_classifier, eps = 0.1, eps_step=0.01, max_iter=20)

correct = 0
eps = 0.1
fc_model.eval()
for j, (images, labels) in enumerate(test_loader):
  images, labels = images.to(device), labels.to(device)
  adv_images = attack.pgd_untargeted(fc_model, images, labels, 20, eps, 0.01)
  logits = fc_model(images.to(device))
  adv_logits = fc_model(adv_images.to(device))
  _, preds = torch.max(logits, 1)
  _, adv_preds = torch.max(adv_logits, 1)
  correct += (preds == labels).sum().item()
  correct += (adv_preds == labels).sum().item()
  # print('Batch [{}/{}]'.format(j+1, len(test_loader)))
fc_model.train()
print('Accuracy = {}%'.format(float(correct) * 100 / 20000))

Accuracy = 53.73%


In [27]:
from art.attacks.inference.membership_inference import MembershipInferenceBlackBox
from art.estimators.classification import PyTorchClassifier

In [ ]:
# optimizer = torch.optim.Adam(fc_model.parameters())
# optimizer = optim.RMSprop(fc_model.parameters(), lr=LR)
# criterion = nn.CrossEntropyLoss()

# Wrap the PyTorch model in ART's PyTorchClassifier
art_classifier = PyTorchClassifier(
    model=fc_model,
    loss=criterion,
    optimizer=optimizer,
    input_shape=(28, 28),
    nb_classes=10
)
attack_train_size = 10000
attack_test_size = 5000

x_train = train_dataset.data
y_train = train_dataset.targets


x_test = test_dataset.data
y_test = test_dataset.targets

attack = MembershipInferenceBlackBox(estimator=art_classifier, attack_model_type="nn")
attack.fit(x_train[:attack_train_size], y_train[:attack_train_size], x_test[:attack_test_size], y_test[:attack_test_size])

mlp_inferred_train_bb = attack.infer(x_train[attack_train_size:], y_train[attack_train_size:])
mlp_inferred_test_bb = attack.infer(x_test[attack_test_size:], y_test[attack_test_size:])

# check accuracy
mlp_train_acc_bb = np.sum(mlp_inferred_train_bb) / len(mlp_inferred_train_bb)
mlp_test_acc_bb = 1 - (np.sum(mlp_inferred_test_bb) / len(mlp_inferred_test_bb))
mlp_acc_bb = (mlp_train_acc_bb * len(mlp_inferred_train_bb) + mlp_test_acc_bb * len(mlp_inferred_test_bb)) / (len(mlp_inferred_train_bb) + len(mlp_inferred_test_bb))


In [16]:
print(f"Members Accuracy: {mlp_train_acc_bb:.4f}")
print(f"Non Members Accuracy {mlp_test_acc_bb:.4f}")
print(f"Attack Accuracy {mlp_acc_bb:.4f}")

Members Accuracy: 0.7896
Non Members Accuracy 0.1430
Attack Accuracy 0.7309


In [43]:

n = 60000
# Create an RDP (Rényi Differential Privacy) accountant

# Compute privacy budget
# epsilon = accountant.get_privacy_spent(delta=1e-5)
accountant.get_epsilon(1e-5)

NameError: name 'asanyarray' is not defined